# Byg en klassifikationsmodel: Lækre asiatiske og indiske retter


## Klassifikatorer for køkken 2

I denne anden lektion om klassifikation vil vi udforske `flere måder` at klassificere kategoriske data på. Vi vil også lære om konsekvenserne ved at vælge én klassifikator frem for en anden.

### [**Quiz før lektionen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Forudsætninger**

Vi antager, at du har gennemført de tidligere lektioner, da vi vil bygge videre på nogle af de begreber, vi tidligere har lært.

Til denne lektion skal vi bruge følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling af R-pakker](https://www.tidyverse.org/packages), der er designet til at gøre datavidenskab hurtigere, nemmere og sjovere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er en [rammeværk af pakker](https://www.tidymodels.org/packages/) til modellering og maskinlæring.

-   `themis`: [themis-pakken](https://themis.tidymodels.org/) tilbyder ekstra opskridt til håndtering af ubalancerede data.

Du kan installere dem med følgende kommando:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativt kan scriptet nedenfor kontrollere, om du har de nødvendige pakker til at gennemføre dette modul, og installere dem for dig, hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Et klassifikationskort**

I vores [forrige lektion](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) forsøgte vi at besvare spørgsmålet: hvordan vælger vi mellem flere modeller? I høj grad afhænger det af dataenes karakteristika og typen af problem, vi ønsker at løse (for eksempel klassifikation eller regression).

Tidligere lærte vi om de forskellige muligheder, du har, når du klassificerer data ved hjælp af Microsofts cheat sheet. Python's Machine Learning-framework, Scikit-learn, tilbyder et lignende, men mere detaljeret cheat sheet, der yderligere kan hjælpe med at indsnævre dine estimators (et andet ord for klassifikatorer):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [besøg dette kort online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) og klik langs stien for at læse dokumentationen.
>
> [Tidymodels reference-siden](https://www.tidymodels.org/find/parsnip/#models) giver også fremragende dokumentation om forskellige typer modeller.

### **Planen** 🗺️

Dette kort er meget nyttigt, når du har et klart overblik over dine data, da du kan 'gå' langs dets stier til en beslutning:

-   Vi har \>50 prøver

-   Vi ønsker at forudsige en kategori

-   Vi har mærkede data

-   Vi har færre end 100K prøver

-   ✨ Vi kan vælge en Linear SVC

-   Hvis det ikke virker, da vi har numeriske data

    -   Kan vi prøve en ✨ KNeighbors Classifier

        -   Hvis det heller ikke virker, prøv ✨ SVC og ✨ Ensemble Classifiers

Dette er en meget nyttig sti at følge. Lad os nu komme i gang med at bruge [tidymodels](https://www.tidymodels.org/) modelleringsrammen: en konsistent og fleksibel samling af R-pakker udviklet til at fremme god statistisk praksis 😊.

## 2. Opdel dataene og håndter ubalancerede datasæt.

Fra vores tidligere lektioner lærte vi, at der var et sæt af fælles ingredienser på tværs af vores køkkener. Derudover var der en ret ulige fordeling i antallet af køkkener.

Vi vil håndtere dette ved at

-   Fjerne de mest almindelige ingredienser, der skaber forvirring mellem forskellige køkkener, ved hjælp af `dplyr::select()`.

-   Bruge en `recipe`, der forbehandler dataene, så de er klar til modellering, ved at anvende en `over-sampling` algoritme.

Vi har allerede kigget på ovenstående i den tidligere lektion, så dette burde være en leg 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Håndtering af ubalancerede data

Ubalancerede data har ofte en negativ indvirkning på modellens ydeevne. Mange modeller fungerer bedst, når antallet af observationer er lige, og har derfor tendens til at kæmpe med ubalancerede data.

Der er primært to måder at håndtere ubalancerede datasæt på:

-   tilføje observationer til minoritetsklassen: `Over-sampling`, f.eks. ved brug af en SMOTE-algoritme, som syntetisk genererer nye eksempler af minoritetsklassen ved hjælp af nærmeste naboer til disse tilfælde.

-   fjerne observationer fra majoritetsklassen: `Under-sampling`

I vores tidligere lektion demonstrerede vi, hvordan man håndterer ubalancerede datasæt ved hjælp af en `recipe`. En recipe kan betragtes som en køreplan, der beskriver, hvilke trin der skal anvendes på et datasæt for at gøre det klar til dataanalyse. I vores tilfælde ønsker vi at have en lige fordeling i antallet af vores køkkener for vores `training set`. Lad os komme i gang.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nu er vi klar til at træne modeller 👩‍💻👨‍💻!

## 3. Ud over multinomial regressionsmodeller

I vores tidligere lektion kiggede vi på multinomial regressionsmodeller. Lad os udforske nogle mere fleksible modeller til klassifikation.

### Support Vector Machines

I forbindelse med klassifikation er `Support Vector Machines` en maskinlæringsteknik, der forsøger at finde et *hyperplan*, som "bedst" adskiller klasserne. Lad os se på et simpelt eksempel:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ adskiller ikke klasserne. H2~ gør det, men kun med en lille margin. H3~ adskiller dem med den maksimale margin.

#### Lineær Support Vector Classifier

Support-Vector clustering (SVC) er en del af Support-Vector maskinerne, som er en familie af ML-teknikker. I SVC vælges hyperplanet, så det korrekt adskiller `de fleste` af træningsobservationerne, men `kan fejlagtigt klassificere` nogle få observationer. Ved at tillade, at nogle punkter er på den forkerte side, bliver SVM mere robust over for outliers og dermed bedre til at generalisere til nye data. Den parameter, der regulerer denne overtrædelse, kaldes `cost`, som har en standardværdi på 1 (se `help("svm_poly")`).

Lad os oprette en lineær SVC ved at sætte `degree = 1` i en polynomisk SVM-model.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nu hvor vi har samlet forbehandlingsskridtene og modelspecifikationen i en *workflow*, kan vi gå videre og træne den lineære SVC og evaluere resultaterne samtidig. For at måle ydeevnen, lad os oprette et sæt af metrikker, der vil evaluere: `accuracy`, `sensitivity`, `Positive Predicted Value` og `F Measure`.

> `augment()` vil tilføje kolonne(r) med forudsigelser til de givne data.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support vector maskinen (SVM) er en udvidelse af support vector klassifikatoren for at kunne håndtere en ikke-lineær grænse mellem klasserne. Grundlæggende bruger SVM'er *kernel-tricket* til at udvide feature-rummet for at tilpasse sig ikke-lineære relationer mellem klasserne. En populær og meget fleksibel kernel-funktion, der bruges af SVM'er, er *Radial basis function.* Lad os se, hvordan den klarer sig på vores data.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Meget bedre 🤩!

> ✅ Se venligst:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> for yderligere læsning.

### Nærmeste nabo-klassifikatorer

*K*-nærmeste nabo (KNN) er en algoritme, hvor hver observation forudsiges baseret på dens *lighed* med andre observationer.

Lad os tilpasse en til vores data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Det ser ud til, at denne model ikke klarer sig så godt. Det er sandsynligt, at ændring af modellens parametre (se `help("nearest_neighbor")`) vil forbedre modellens ydeevne. Sørg for at prøve det.

> ✅ Se venligst:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> for at lære mere om *K*-Nearest Neighbors klassifikatorer.

### Ensemble klassifikatorer

Ensemble-algoritmer fungerer ved at kombinere flere basismodeller for at producere en optimal model enten ved:

`bagging`: anvendelse af en *gennemsnitsfunktion* på en samling af basismodeller

`boosting`: opbygning af en sekvens af modeller, der bygger på hinanden for at forbedre den prædiktive ydeevne.

Lad os starte med at prøve en Random Forest-model, som opbygger en stor samling af beslutningstræer og derefter anvender en gennemsnitsfunktion for at skabe en bedre samlet model.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Godt klaret 👏!

Lad os også eksperimentere med en Boosted Tree-model.

Boosted Tree definerer en ensemblemetode, der skaber en række sekventielle beslutningstræer, hvor hvert træ afhænger af resultaterne fra de tidligere træer i et forsøg på gradvist at reducere fejlen. Den fokuserer på vægten af forkert klassificerede elementer og justerer tilpasningen for den næste klassifikator for at rette op.

Der er forskellige måder at tilpasse denne model på (se `help("boost_tree")`). I dette eksempel vil vi tilpasse Boosted Trees via `xgboost`-motoren.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Se:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Udforsker AdaBoost-modellen, som er et godt alternativ til xgboost.
>
> for at lære mere om Ensemble-klassifikatorer.

## 4. Ekstra - sammenligning af flere modeller

Vi har tilpasset en hel del modeller i denne øvelse 🙌. Det kan blive tidskrævende eller besværligt at oprette mange workflows fra forskellige sæt af preprocessorer og/eller modelspecifikationer og derefter beregne præstationsmålinger én efter én.

Lad os se, om vi kan løse dette ved at oprette en funktion, der tilpasser en liste af workflows på træningssættet og derefter returnerer præstationsmålingerne baseret på test-sættet. Vi kommer til at bruge `map()` og `map_dfr()` fra [purrr](https://purrr.tidyverse.org/) pakken til at anvende funktioner på hvert element i en liste.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) funktioner giver dig mulighed for at erstatte mange for-løkker med kode, der både er mere kortfattet og lettere at læse. Det bedste sted at lære om [`map()`](https://purrr.tidyverse.org/reference/map.html) funktionerne er [iteration-kapitlet](http://r4ds.had.co.nz/iteration.html) i R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) pakken giver brugerne mulighed for at oprette og nemt tilpasse et stort antal modeller, men den er primært designet til at arbejde med genprøvningsteknikker som f.eks. `cross-validation`, en metode vi endnu ikke har dækket.

## **🚀Udfordring**

Hver af disse teknikker har et stort antal parametre, som du kan justere, for eksempel `cost` i SVM'er, `neighbors` i KNN, `mtry` (Tilfældigt Valgte Predictors) i Random Forest.

Undersøg standardparametrene for hver enkelt og overvej, hvad det ville betyde for modellens kvalitet at justere disse parametre.

For at finde mere information om en bestemt model og dens parametre, brug: `help("model")`, f.eks. `help("rand_forest")`.

> I praksis *estimerer* vi normalt de *bedste værdier* for disse ved at træne mange modeller på et `simuleret datasæt` og måle, hvor godt alle disse modeller præsterer. Denne proces kaldes **tuning**.

### [**Quiz efter forelæsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Gennemgang & Selvstudie**

Der er mange fagudtryk i disse lektioner, så tag et øjeblik til at gennemgå [denne liste](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) med nyttig terminologi!

#### TAK TIL:

[`Allison Horst`](https://twitter.com/allison_horst/) for at skabe de fantastiske illustrationer, der gør R mere imødekommende og engagerende. Find flere illustrationer i hendes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) og [Jen Looper](https://www.twitter.com/jenlooper) for at skabe den originale Python-version af dette modul ♥️

God læring,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Illustration af @allison_horst</figcaption>



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
